In [472]:
import pandas as pd
import os

In [473]:
lista = [i for i in os.listdir() if 'xlsx' in i]

In [474]:
dfs = []

for i in lista:
    df = pd.read_excel(i, skiprows=3).iloc[:-1]
    print(df)
    df['mes'] = i.split('.')[0]
    dfs.append(df)


           UEJ                              NOMBRE UEJ                 RUBRO  \
0     01-01-01                  SENADO DE LA REPUBLICA            A-01-01-01   
1     01-01-01                  SENADO DE LA REPUBLICA            A-01-01-02   
2     01-01-01                  SENADO DE LA REPUBLICA            A-01-01-03   
3     01-01-01                  SENADO DE LA REPUBLICA                  A-02   
4     01-01-01                  SENADO DE LA REPUBLICA            A-03-02-02   
...        ...                                     ...                   ...   
3778  46-04-00  INSTITUTO NACIONAL PARA CIEGOS (INCI)   C-4601-1500-2-707010   
3779  46-04-00  INSTITUTO NACIONAL PARA CIEGOS (INCI)   C-4601-1500-2-707010   
3780  46-04-00  INSTITUTO NACIONAL PARA CIEGOS (INCI)   C-4699-1500-2-707010   
3781  46-04-00  INSTITUTO NACIONAL PARA CIEGOS (INCI)   C-4699-1500-2-707010   
3782  46-04-00  INSTITUTO NACIONAL PARA CIEGOS (INCI)   C-4699-1500-2-707010   

     TIPO     CTA  SUB\nCTA  OBJ     OR

In [475]:
df = pd.concat(dfs[:-1], ignore_index=True)

In [476]:
import json

with open("dic_entidad.json", "r") as f:
    dic_entidad = json.load(f)

with open("dic_sector.json", "r") as f:
    dic_sector = json.load(f)

with open("dic_unidad.json", "r") as f:
    dic_unidad = json.load(f)

In [477]:
prog = pd.read_excel("programacion_2025.xlsx", 
                     sheet_name="Programación de gastos",
                     skiprows=6)

In [478]:
prog = prog[~prog['Cuenta'].isin(['A- PRESUPUESTO DE FUNCIONAMIENTO',
                          'B- PRESUPUESTO DE SERVICIO DE LA DEUDA PÚBLICA',
                          'C - PRESUPUESTO DE INVERSIÓN',
                          'C'])].drop(index=1).reset_index(drop=True)
prog.columns = ['Cuenta_n', 'Subcuenta_n', 'Objeto_n', 'Ordinal_n',
                'Cuenta', 'Subcuenta', 'Objeto', 'Ordinal']

In [479]:
dic_cuentas = dict(prog[['Cuenta_n', 'Cuenta']].dropna().set_index('Cuenta_n').to_records())
prog['Cuenta'] = prog['Cuenta_n'].map(dic_cuentas)

In [480]:
prog['Subcuenta_n'] = prog['Subcuenta_n'].astype(str)
prog['Objeto_n'] = prog['Objeto_n'].astype(str)

In [481]:
for i in prog['Cuenta_n'].unique():
    filtro_cuenta = prog[prog['Cuenta_n'] == i]
    dic_subcuentas = dict(filtro_cuenta[['Subcuenta_n', 'Subcuenta']].dropna().set_index('Subcuenta_n').to_records())
    filtro_cuenta['Subcuenta'] = filtro_cuenta['Subcuenta_n'].map(dic_subcuentas)
    prog.loc[prog['Cuenta_n'] == i] = filtro_cuenta
    for j in filtro_cuenta['Subcuenta_n'].unique():
        filtro_subcuenta = filtro_cuenta[filtro_cuenta['Subcuenta_n'] == j]
        dic_objetos = dict(filtro_cuenta[['Objeto_n', 'Objeto']].dropna().set_index('Objeto_n').to_records())
        filtro_subcuenta['Objeto'] = filtro_cuenta['Objeto_n'].map(dic_objetos)
        prog.loc[(prog['Subcuenta_n'] == j) & (prog['Cuenta_n'] == i)] = filtro_subcuenta
        for k in filtro_subcuenta['Objeto_n'].unique():
            filtro_objeto = filtro_subcuenta[filtro_subcuenta['Objeto_n'] == k]
            dic_ordinal = dict(filtro_cuenta[['Ordinal_n', 'Ordinal']].dropna().set_index('Ordinal_n').to_records())
            filtro_objeto['Ordinal'] = filtro_cuenta['Ordinal_n'].map(dic_ordinal)
            prog.loc[(prog['Subcuenta_n'] == j) & (prog['Cuenta_n'] == i) & (prog['Objeto_n'] == k)] = filtro_objeto           



C:\Users\user\AppData\Local\Temp\ipykernel_9204\3370647205.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_cuenta['Subcuenta'] = filtro_cuenta['Subcuenta_n'].map(dic_subcuentas)
C:\Users\user\AppData\Local\Temp\ipykernel_9204\3370647205.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_subcuenta['Objeto'] = filtro_cuenta['Objeto_n'].map(dic_objetos)
C:\Users\user\AppData\Local\Temp\ipykernel_9204\3370647205.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [482]:
df['Código de entidad'] = df['UEJ'].str.split('-').str[:2].str.join('').astype(int).astype(str)
df['Código de sector'] = df['UEJ'].str.split('-').str[0].astype(int).astype(str)
df['Código de unidad'] = df['UEJ'].str.split('-').str.join('').astype(int).astype(str)
# nombre de sector

df['Sector'] = df['Código de sector'].map(dic_sector)
# nombre de entidad

df['Entidad'] = df['Código de entidad'].map(dic_entidad)
# nombre de unidad
df['Unidad'] = df['NOMBRE UEJ'].str.capitalize()



# tipo de gasto

tipo_gasto = {'A': 'Funcionamiento',
             'B': 'Deuda',
             'C': 'Inversión'}

df['Tipo de gasto'] = df['TIPO'].map(tipo_gasto)
# código de rubro
df['Código de rubro'] = df['RUBRO']
# nombre de rubro
df['Nombre de rubro'] = df['DESCRIPCION']

# nombres de las cuentas

# nombres de las subcuentas

# nombres de objetos

# nombres de ordinal

# número de mes
mes = {'enero': 1,
       'febrero': 2,
       'marzo': 3,
       'abril': 4,
       'mayo': 5,
       'junio': 6,
       'julio': 7,
       'agosto': 8}
df['mes_num'] = df['mes'].map(mes)
# ratios de seguimiento
# eliminar columnas


In [483]:
df['DESCRIPCION']



0                                                  SALARIO
1                    CONTRIBUCIONES INHERENTES A LA NÓMINA
2        REMUNERACIONES NO CONSTITUTIVAS DE FACTOR SALA...
3                       ADQUISICIÓN DE BIENES  Y SERVICIOS
4                         A ORGANIZACIONES INTERNACIONALES
                               ...                        
30650    7. ACTORES DIFERENCIALES PARA EL CAMBIO / 1. U...
30651    7. ACTORES DIFERENCIALES PARA EL CAMBIO / 1. U...
30652    7. ACTORES DIFERENCIALES PARA EL CAMBIO / 1. U...
30653    7. ACTORES DIFERENCIALES PARA EL CAMBIO / 1. U...
30654    7. ACTORES DIFERENCIALES PARA EL CAMBIO / 1. U...
Name: DESCRIPCION, Length: 30655, dtype: object

In [484]:
df[['UEJ', 'NOMBRE UEJ', 'RUBRO', 'TIPO', 'CTA', 'SUB\nCTA', 'OBJ', 'ORD',
       'SOR\nORD', 'ITEM', 'SUB\nITEM', 'SUB\nITEM 2']].isna().sum()

UEJ                0
NOMBRE UEJ         0
RUBRO              0
TIPO               0
CTA                0
SUB\nCTA        2054
OBJ             5014
ORD            12836
SOR\nORD       30655
ITEM           30655
SUB\nITEM      30655
SUB\nITEM 2    30655
dtype: int64

In [485]:
import numpy as np

def f_cuenta(row):
    cuenta = row['CTA']

    if len(str(int(cuenta))) == 1:
        return f"0{int(cuenta)}"
    return str(int(cuenta))

def f_subcuenta(row):
    subcuenta = row['SUB\nCTA']
    try:
        if len(str(int(subcuenta))) == 1:
            return f"0{int(subcuenta)}"
        elif len(str(int(subcuenta))) == 2:
            return str(int(subcuenta))
        else:
            return str(int(subcuenta))
    except:
        return np.nan
    
def f_objeto(row):
    objeto = row['OBJ']

    try:
        if len(str(int(objeto))) == 1:
            return f"0{int(objeto)}"
        return str(int(objeto))

    except:
        return np.nan
def f_ordinal(row):
    ordinal = row['ORD']

    try:
        if len(str(int(ordinal))) == 1:
            return f"00{int(ordinal)}"
        elif len(str(int(ordinal))) == 2:
            return f"0{int(ordinal)}"
        else:
            return str(int(ordinal))


    except:
        return np.nan
    


In [486]:
df['Cuenta_n'] = df.apply(f_cuenta, axis=1) 
df['Subcuenta_n'] = df.apply(f_subcuenta, axis=1)
df['Objeto_n'] = df.apply(f_objeto, axis=1)
df['Ordinal_n'] = df.apply(f_ordinal, axis=1)

df['Cuenta'] = df['Cuenta_n'].map(dic_cuentas).fillna(df['Cuenta_n'])
df['Subcuenta'] = np.nan
df['Objeto'] = np.nan
df['Ordinal'] = np.nan

In [487]:
df.columns

Index(['UEJ', 'NOMBRE UEJ', 'RUBRO', 'TIPO', 'CTA', 'SUB\nCTA', 'OBJ', 'ORD',
       'SOR\nORD', 'ITEM', 'SUB\nITEM', 'SUB\nITEM 2', 'FUENTE', 'REC', 'SIT',
       'DESCRIPCION', 'APR. INICIAL', 'APR. ADICIONADA', 'APR. REDUCIDA',
       'APR. VIGENTE', 'APR BLOQUEADA', 'CDP', 'APR. DISPONIBLE', 'COMPROMISO',
       'OBLIGACION', 'ORDEN PAGO', 'PAGOS', 'mes', 'Código de entidad',
       'Código de sector', 'Código de unidad', 'Sector', 'Entidad', 'Unidad',
       'Tipo de gasto', 'Código de rubro', 'Nombre de rubro', 'mes_num',
       'Cuenta_n', 'Subcuenta_n', 'Objeto_n', 'Ordinal_n', 'Cuenta',
       'Subcuenta', 'Objeto', 'Ordinal'],
      dtype='object')

In [488]:
for cuenta in prog['Cuenta_n'].unique():
    print('cuenta', cuenta)
    prog_cuenta = prog[prog['Cuenta_n'] == cuenta]
    fil_cuenta = df[df['Cuenta_n'] == cuenta]
    dic_subcuentas = dict(prog_cuenta[['Subcuenta_n', 'Subcuenta']].dropna().set_index('Subcuenta_n').to_records())
    df.loc[fil_cuenta.index, 'Subcuenta'] = fil_cuenta['Subcuenta_n'].map(dic_subcuentas)
    print('dic_subcuentas', dic_subcuentas)
    for subcuenta in prog_cuenta['Subcuenta_n'].unique():
        print('\tsubcuenta', subcuenta)

        prog_sub = prog_cuenta[prog_cuenta['Subcuenta_n'] == subcuenta]
        fil_sub = fil_cuenta[fil_cuenta['Subcuenta_n'] == subcuenta]
        dic_objetos = dict(prog_sub[['Objeto_n', 'Objeto']].dropna().set_index('Objeto_n').to_records())
        df.loc[fil_sub.index, 'Objeto'] = fil_sub['Objeto_n'].map(dic_objetos)
        print("\t", fil_sub)
        print('\tdic_objetos', dic_objetos)
        print("\t",fil_sub['Objeto_n'].map(dic_objetos))
        for objeto in prog_sub['Objeto_n'].unique():
            print('\t\tobjeto', objeto)
            prog_obj = prog_sub[prog_sub['Objeto_n'] == objeto]
            fil_obj = fil_sub[fil_sub['Objeto_n'] == objeto]
            dic_ord = dict(prog_obj[['Ordinal_n', 'Ordinal']].dropna().set_index('Ordinal_n').to_records())
            df.loc[fil_obj.index, 'Ordinal'] = fil_obj['Ordinal_n'].map(dic_ord)
            print('\t\tdic_ord', dic_ord)
    print('---' * 10)



cuenta 01
dic_subcuentas {'01': 'PLANTA DE PERSONAL PERMANENTE', '02': 'PERSONAL SUPERNUMERARIO Y PLANTA TEMPORAL'}
	subcuenta nan
	 Empty DataFrame
Columns: [UEJ, NOMBRE UEJ, RUBRO, TIPO, CTA, SUB
CTA, OBJ, ORD, SOR
ORD, ITEM, SUB
ITEM, SUB
ITEM 2, FUENTE, REC, SIT, DESCRIPCION, APR. INICIAL, APR. ADICIONADA, APR. REDUCIDA, APR. VIGENTE, APR BLOQUEADA, CDP, APR. DISPONIBLE, COMPROMISO, OBLIGACION, ORDEN PAGO, PAGOS, mes, Código de entidad, Código de sector, Código de unidad, Sector, Entidad, Unidad, Tipo de gasto, Código de rubro, Nombre de rubro, mes_num, Cuenta_n, Subcuenta_n, Objeto_n, Ordinal_n, Cuenta, Subcuenta, Objeto, Ordinal]
Index: []

[0 rows x 46 columns]
	dic_objetos {}
	 Series([], Name: Objeto_n, dtype: float64)
		objeto nan
		dic_ord {}
	subcuenta 01
	             UEJ                              NOMBRE UEJ       RUBRO TIPO  CTA  \
0      01-01-01                  SENADO DE LA REPUBLICA  A-01-01-01    A  1.0   
1      01-01-01                  SENADO DE LA REPUBLICA  A

C:\Users\user\AppData\Local\Temp\ipykernel_9204\1614040797.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['PLANTA DE PERSONAL PERMANENTE' 'PLANTA DE PERSONAL PERMANENTE'
 'PLANTA DE PERSONAL PERMANENTE' ... 'PLANTA DE PERSONAL PERMANENTE'
 'PLANTA DE PERSONAL PERMANENTE' 'PLANTA DE PERSONAL PERMANENTE']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[fil_cuenta.index, 'Subcuenta'] = fil_cuenta['Subcuenta_n'].map(dic_subcuentas)
C:\Users\user\AppData\Local\Temp\ipykernel_9204\1614040797.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SALARIO' 'CONTRIBUCIONES INHERENTES A LA NÓMINA'
 'REMUNERACIONES NO CONSTITUTIVAS DE FACTOR SALARIAL' ... 'SALARIO'
 'CONTRIBUCIONES INHERENTES A LA NÓMINA'
 'REMUNERACIONES NO CONSTITUTIVAS DE FACTOR SALARIAL']' has dtype incompa

            UEJ                                         NOMBRE UEJ  \
4      01-01-01                             SENADO DE LA REPUBLICA   
68     02-09-00  AGENCIA PRESIDENCIAL DE COOPERACION INTERNACIO...   
193    04-01-01  DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADÍ...   
256    05-03-00  ESCUELA SUPERIOR DE ADMINISTRACIÓN PÚBLICA (ESAP)   
287    11-02-00  FONDO ROTATORIO DEL MINISTERIO DE RELACIONES E...   
...         ...                                                ...   
30019  35-03-00           SUPERINTENDENCIA DE INDUSTRIA Y COMERCIO   
30052  35-05-00             INSTITUTO NACIONAL DE METROLOGIA - INM   
30081  36-01-01           MINISTERIO DEL TRABAJO - GESTION GENERAL   
30185  36-02-00            SERVICIO NACIONAL DE APRENDIZAJE (SENA)   
30323  37-03-00            DIRECCION NACIONAL DEL DERECHO DE AUTOR   

            RUBRO TIPO  CTA  SUB\nCTA  OBJ  ORD  SOR\nORD  ITEM  ...  \
4      A-03-02-02    A  3.0       2.0  2.0  NaN       NaN   NaN  ...   
68     A-03-02-

In [489]:
df.loc[(df['Ordinal_n'] == '001') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '001') & (df['Ordinal'].isna()), 'Nombre de rubro']
df.loc[(df['Ordinal_n'] == '088') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '088') & (df['Ordinal'].isna()), 'Nombre de rubro']
df.loc[(df['Ordinal_n'] == '073') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '073') & (df['Ordinal'].isna()), 'Nombre de rubro']
df.loc[(df['Ordinal_n'] == '072') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '072') & (df['Ordinal'].isna()), 'Nombre de rubro']
df.loc[(df['Ordinal_n'] == '074') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '074') & (df['Ordinal'].isna()), 'Nombre de rubro']
df.loc[(df['Ordinal_n'] == '089') & (df['Ordinal'].isna()), 'Ordinal'] = df.loc[(df['Ordinal_n'] == '089') & (df['Ordinal'].isna()), 'Nombre de rubro']


In [494]:
df[df['TIPO'] == 'A'][['Cuenta_n', 'Subcuenta_n', 'Objeto_n', 'Ordinal_n']].isna().sum()

Cuenta_n           0
Subcuenta_n     2054
Objeto_n        5014
Ordinal_n      12598
dtype: int64

In [495]:
df[df['TIPO'] == 'A'][['Cuenta', 'Subcuenta', 'Objeto', 'Ordinal']].isna().sum()

Cuenta           0
Subcuenta     2054
Objeto        5014
Ordinal      12598
dtype: int64

In [498]:
df.drop(columns=['RUBRO', 'DESCRIPCION','UEJ', 'NOMBRE UEJ', 'SOR\nORD', 'ITEM', 'SUB\nITEM', 'SUB\nITEM 2',], inplace=True)

df.to_csv('ejecucion_agosto.csv', index=False)